# Getting Started with DuckDB

DuckDB is a modern, high-performance, in-memory analytical database management system (DBMS) designed to support 
complex analytical queries. It is a relational (table-oriented) DBMS that supports the Structured Query Language 
(SQL). DuckDB combines the simplicity and ease of use of SQLite with the high-performance capabilities required 
for analytical workloads, making it an excellent choice for data scientists and analysts.

Key Features >>

Simple operation: DuckDB is serverless, has no external dependencies, and is embedded within a host process. 
This makes it easy to install and deploy, requiring only a C++11 compiler for building.

Feature-rich: It supports extensive SQL data management features. DuckDB also offers deep integration with Python 
and R, making it suitable for data science and interactive data analysis.

Fast analytical queries: DuckDB uses a columnar-vectorized query execution engine optimized for analytics, enabling
parallel query processing and efficient handling of large datasets. 

Free and open source: It is released under the permissive MIT License, making it free to use and open-source. 

Portability: With no external dependencies, DuckDB is highly portable and can run on various operating systems 
(Linux, macOS, Windows) and CPU architectures (x86, ARM). It can even run in web browsers using DuckDB-Wasm.

Extensibility: DuckDB supports a flexible extension mechanism, allowing the addition of new data types, 
functions, file formats, and SQL syntax. 

Thorough testing: It undergoes intensive testing using Continuous Integration, with a test suite containing 
millions of queries. This ensures stability and reliability across different platforms and compilers.

In [ ]:
pip install duckdb --upgrade

In this section, we will learn to set up DuckDB, load CSV files, perform data analysis, and learn about relations and query functions. 

We will start by installing the DuckDB Python package. 

This line of code is a command used in the terminal or command prompt, not in a Python script.

pip is a package installer for Python. It's used to install and manage software packages/libraries.

install is a command that tells pip to install a package.

duckdb is the name of the package that pip is being told to install.

--upgrade is an optional argument that tells pip to upgrade the package if it's already installed.

# Creating the DuckDB database

In [ ]:
import duckdb
con = duckdb.connect("datacamp.duckdb")

To create the persistent database, you just have to use the connect function and provide it with the database name. 

The code starts by importing the duckdb module, which is a fast analytical database written in C++.

Then, it establishes a connection to a DuckDB database named "datacamp.duckdb" using the connect method.

It will create a database base file in your local directory.

In [ ]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS 
    SELECT * FROM read_csv('bank-marketing.csv')
""")
con.execute("SHOW ALL TABLES").fetchdf()

We will load a CSV file and create a "bank" table. The dataset we are using is available on DataLab and is called Bank Marketing. It consists of direct marketing campaigns by a Portuguese banking institution using phone calls.

To load the CSV file, you have to create a Table first using SQL and then use the read_csv() function within the SQL script to load the file. It is that simple. 

We will then validate our table by executing the SQL script that shows all of the tables within the database and using the fetchdf function to display the result as a pandas DataFrame. 

The con.execute() function is used to execute SQL commands.

The first con.execute() runs a CREATE TABLE SQL command.

IF NOT EXISTS checks if the table 'bank' already exists, and if it does, the command is ignored.

AS SELECT * FROM read_csv('bank-marketing.csv') creates the 'bank' table using data from the CSV file.

The read_csv() function reads the 'bank-marketing.csv' file.

The second con.execute() runs a SHOW ALL TABLES SQL command.

fetchdf() fetches the result of the SHOW ALL TABLES command as a DataFrame.

In [ ]:
con.execute("SELECT * FROM bank WHERE duration < 100 LIMIT 5").fetchdf()

Now that we have successfully created our first table, we will run a beginner-level query to analyze the data and display the result as a DataFrame.

con.execute() is a method that runs the SQL query enclosed in the parentheses.

"SELECT * FROM bank WHERE duration &lt; 100 LIMIT 5" is the SQL query being executed.

SELECT * FROM bank selects all columns from the 'bank' table.

WHERE duration &lt; 100 filters the data to only include rows where the 'duration' is less than 100.

LIMIT 5 restricts the output to the first 5 rows that match the condition.

.fetchdf() is a method that fetches the result of the query and returns it as a DataFrame.

# DuckDB Relations

In [ ]:
bank_duck = duckdb.read_csv("bank-marketing.csv",sep=";")
bank_duck.filter("duration < 100").limit(3).df()

DuckDB relations are essentially tables that can be queried using the Relational API. This API allows for the chaining of various query operations on data sources like Pandas DataFrames. Instead of using SQL queries, you will by chaining together various Python functions to analyze the data. 

For example, we will load a CSV file to create the DuckDB relation. To analyze the table, you can chain the filter and limit functions.

The code uses the duckdb library, which is a high-performance analytical database system.

duckdb.read_csv("bank-marketing.csv",sep=";") reads a CSV file named "bank-marketing.csv" using a semicolon as a separator.

The read CSV file is stored in the bank_duck variable.

bank_duck.filter("duration &lt; 100") filters the data to only include rows where the "duration" column is less than 100.

.limit(3) further limits the output to the first 3 rows of the filtered data.

.df() converts the result into a pandas DataFrame, which is a two-dimensional, size-mutable, and heterogeneous tabular data structure.

In [ ]:
rel = con.table("bank")
rel.columns

We can also create relations by loading the table from the DuckDB database. 

The code is using a database connection object con to access a table named "bank".

rel = con.table("bank") is accessing the "bank" table from the database and storing it in rel.

rel.columns is then used to display the column names of the "bank" table.

In [ ]:
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

Let’s write a relation that uses multiple functions to analyze the data. 

rel.filter("duration &lt; 100") filters the data to include only rows where the 'duration' value is less than 100.

.project("job,education,loan") selects only the 'job', 'education', and 'loan' columns from the filtered data.

.order("job") sorts the data based on the 'job' column in ascending order.

.limit(3) limits the output to the first 3 rows of the sorted data.

.df() converts the result into a pandas DataFrame.

We have three rows and columns sorted by job and filtered by duration column.

# DuckDB Query Function

In [ ]:
res = duckdb.query("""SELECT 
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM 
                            'bank-marketing.csv'
                        WHERE 
                            age > 30
                        GROUP BY 
                            job
                        ORDER BY 
                            total_clients_contacted DESC;""")
res.df()

The DuckDB query function allows SQL queries to be executed within the database, returning results that can be converted into various formats for further analysis.

In the code example, we are running the SQL query to find out the job titles of clients over the age of 30, count the number of clients contacted for each job, and calculate the average duration of the campaign.

The code is written in SQL and is executed using the DuckDB Python library.

res = duckdb.query("""...""") is running an SQL query and storing the result in the variable res.

SELECT job, COUNT(*) AS total_clients_contacted, AVG(duration) AS avg_campaign_duration is selecting the 'job' column, the count of rows (renamed as 'total_clients_contacted'), and the average of the 'duration' column (renamed as 'avg_campaign_duration').

FROM 'bank-marketing.csv' is specifying the data source, a CSV file named 'bank-marketing.csv'.

WHERE age &gt; 30 is filtering the data to only include rows where the 'age' column is greater than 30.

GROUP BY job is grouping the selected data by the 'job' column.

ORDER BY total_clients_contacted DESC is ordering the result in descending order based on the 'total_clients_contacted' column.

res.df() is converting the result into a pandas DataFrame for easier manipulation and analysis.

In [ ]:
con.close()

We will now close the connection to the database and release any resources associated with that connection, preventing potential memory and file handle leaks.

This is a single line of code that closes the connection to a database.

con is the variable representing the database connection.

.close() is a method used to terminate the connection.

# Building a RAG Application with DuckDB

In the first project, we will learn to build an RAG application with LlamaIndex and use DuckDB as a Vector database and retriever. 

In [ ]:
%%capture
%pip install duckdb
%pip install llama-index
%pip install llama-index-vector-stores-duckdb

Install all the necessary Python packages that will be used to create and retrieve the index. 

The %%capture command is a Jupyter notebook magic command that suppresses the output of the cell.

The %pip install duckdb command installs the DuckDB library, an in-memory analytical database.

The %pip install llama-index command installs the Llama Index library, a Python library for indexing.

The %pip install llama-index-vector-stores-duckdb command installs a specific component of the Llama Index library that integrates with DuckDB.

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core import StorageContext

from IPython.display import Markdown, display

Import the necessary Python package with the functions.

The code imports the VectorStoreIndex and SimpleDirectoryReader classes from the llama_index.core module.

It then imports the DuckDBVectorStore class from the llama_index.vector_stores.duckdb module.

The StorageContext class is also imported from the llama_index.core module.

Finally, the Markdown and display functions are imported from the IPython.display module.

# Setting up GPT-4o and Embedding Model

In [ ]:
import os
from llama_index.llms.openai import OpenAI

os.environ["OPEN_AI_API_KEY"] = "OPEN_AI_API_KEY"
llm = OpenAI(model="gpt-4", api_key="OPEN_AI_API_KEY")

For a language model, we will use the latest GPT4o model and the OpenAI API. To create the large language model (LLM) client, you just have to provide a model name and API key. 

The code begins by importing the 'os' module, which provides functions for interacting with the operating system.

Then, it imports the 'OpenAI' class from the 'llama_index.llms.openai' module.

It creates an instance 'llm' of the 'OpenAI' class.

The 'OpenAI' class is initialized with two parameters: 'model' and 'api_key'.

The 'model' parameter is set to "gpt-4o", which is the name of the model to be used.

The 'api_key' parameter is retrieved from the environment variables using 'os.environ["OPENAI_API_KEY"]'.

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
)

Then, we will create the embed model client using the OpenAI text-embedding-3-small model. 

The code starts with importing the OpenAIEmbedding class from the llama_index.embeddings.openai module.

Then, it creates an instance of the OpenAIEmbedding class named embed_model.

While creating the instance, it passes "text-embedding-3-small" as an argument to the model parameter of the OpenAIEmbedding class.

The embed_model instance can now be used to generate embeddings for text data using the specified OpenAI model.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

We will make OpenAI LLM and Embedding models global for all LlamaIndex functions to use. In short, these models will be set as default.

The code begins with importing the Settings class from the llama_index.core module.

The Settings class is likely a configuration class for the llama_index package.

The llm object is assigned to the llm attribute of the Settings class.

The embed_model object is assigned to the embed_model attribute of the Settings class.

These assignments are probably used to configure the behavior of the llama_index package.

In [ ]:
documents = SimpleDirectoryReader("Data").load_data()

For our project, we will load the PDF files from the data folder. These PDF files are tutorials from DataCamp that are saved as PDF files using the browser’s print function. 

Provide the folder directory to the SimpleDirectoryReader function and load the data. 

The code is written in Python and it's using a class named SimpleDirectoryReader.

SimpleDirectoryReader("Data") creates an instance of the SimpleDirectoryReader class.

The string "Data" is passed as an argument, which is likely the name of the directory to be read.

The method load_data() is called on the instance, which presumably loads the data from the specified directory.

The loaded data is then stored in the variable documents.

# Using DuckDB as a vector database

In [ ]:
vector_store = DuckDBVectorStore(database_name = "datacamp.duckdb",table_name = "blog",persist_dir="./", embed_dim=1536)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Then, create the vector store called “blog” using an existing database called “datacamp.duckdb.” After that, convert the PDF's data into embeddings and store them in the vector store. 

The first line creates an instance of DuckDBVectorStore with specified database and table names, a directory for persistence, and embedding dimension.

The second line creates a StorageContext using the default settings and the previously created vector_store.

The final line creates a VectorStoreIndex from a list of documents, using the previously created storage_context.

In [ ]:
import duckdb
con = duckdb.connect("datacamp.duckdb")

con.execute("SHOW ALL TABLES").fetchdf()

To check if our vector store was successfully created, we will connect the database using the DuckDB Python API and run the SQL query to display all the tables in the database. 

The code starts by importing the duckdb module, which is a high-performance analytical database system.

The duckdb.connect("datacamp.duckdb") line establishes a connection to a DuckDB database file named "datacamp.duckdb".

con.execute("SHOW ALL TABLES") sends a SQL command to the database to retrieve all table names.

The .fetchdf() method is then used to fetch the result of the SQL command as a pandas DataFrame.

We have two tables: a “bank” promotional table and a “blog” table, which is a vector store. The “blog” table has an “embedding” column where all the embeddings are stored.

# Creating a simple RAG application

Convert the index into the query engine, which will automatically first search the vector database for similar documents and use the additional context to generate the response. 

To test the RAG query engine, we will ask the question about the tutorial. 

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who wrote 'GitHub Actions and MakeFile: A Hands-on Introduction'?")
display(Markdown(f"<b>{response}</b>"))

The first line creates a query engine from an index object using the as_query_engine() method.

The second line uses the query() method of the query engine to search for the author of a specific article.

The third line uses the Markdown function to format the response in bold, and display function to show it.

The output of the query is "The author of "GitHub Actions and MakeFile: A Hands-on Introduction" is Abid Ali Awan." which is the correct answer 

# Creating a RAG chatbot with memory

Now, let’s create an advanced RAG application that uses the conversation history to generate the response. For that, we have to create a chat memory buffer and then a chat engine with memory, LLM, and vector store retriever. 

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm
)

response = chat_engine.chat(
    "What is the easiest way of finetuning the Llama 3 model? Please provide step-by-step instructions."
)

display(Markdown(response.response))

The code starts by importing ChatMemoryBuffer from llama_index.core.memory.

It also imports CondensePlusContextChatEngine from llama_index.core.chat_engine.

A ChatMemoryBuffer object is created with a token limit of 3900.

A CondensePlusContextChatEngine object is created using default settings, the previously created memory buffer, and llm.

The chat method of chat_engine is called with a question about fine-tuning the Llama 3 model.

The response from the chat engine is displayed using the Markdown function.

We asked the chat engine how to fine-tune the Llama 3 model, and it used the vector store to give a highly accurate answer.

In [ ]:
response = chat_engine.chat(
    "Could you please provide more details about the Post Fine-Tuning Steps?"
)
display(Markdown(response.response))

To check if the memory buffer is working correctly, we will ask a follow-up question. 

The code is using a chat engine, which is likely an AI model, to generate a response to a given input.

The method chat_engine.chat() is called with a string argument, which is the question to be asked.

The response from the chat engine is stored in the variable response.

Markdown(response.response) is used to format the response text in Markdown, a lightweight markup language.

The display() function is then used to display the formatted text in the output.

The chat engine remembered the previous conversation and responded accordingly. 

# Building a DuckDB SQL Query Engine Using an LLM

In the second project, we will use DuckDB as an SQL query engine. This involves integrating the database engine with the GPT-4o model to generate natural language responses to questions about the database. 

In [ ]:
%pip install duckdb-engine -q

Install duckdb-engine to create a database engine using SQLAlchemy.

The code is using the %pip command, which is a magic command in Jupyter notebooks.

The install argument tells pip to install a package.

duckdb-engine is the name of the package that is being installed.

The -q flag is used to run the command in quiet mode, reducing the output displayed.

# Loading the DuckDB database

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("duckdb:///datacamp.duckdb")
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM bank LIMIT 3")
    print(cursor.fetchall())

We will load the DuckDB database using the create_engine function and then write a simple SQL query to check whether it is successfully loaded.

The code begins by importing the create_engine function from the sqlalchemy module.

create_engine is then used to create an engine that connects to a DuckDB database file named datacamp.duckdb.

with engine.connect() is used to establish a connection to the database.

Inside the with block, connection.exec_driver_sql("SELECT * FROM bank LIMIT 3") is executed.

This SQL query selects all columns from the first three rows of the 'bank' table in the database.

The results of the query are fetched with cursor.fetchall() and then printed to the console.

Prefect. Our DuckDB database engine is ready to be used. 

In [ ]:
[(56, 'housemaid', 'married', 'basic.4y', 'no', 'no', 'no', 'telephone', 'may', 'mon', 261, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, 'no'), (57, 'services', 'married', 'high.school', 'unknown', 'no', 'no', 'telephone', 'may', 'mon', 149, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, 'no'), (37, 'services', 'married', 'high.school', 'no', 'yes', 'no', 'telephone', 'may', 'mon', 226, 1, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.857, 5191.0, 'no')]

The provided Python code is a list of tuples.

Each tuple in the list contains 21 elements.

These elements could represent different attributes of a dataset, such as age, job, marital status, education, etc.

The data types of the elements in the tuples vary, including integers, strings, and floating-point numbers.

The data seems to be structured and could be used for further analysis or processing in a program.

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["bank"])

Now, we have to create a database Tool using the SQLDatabase function. Provide it with an engine object and table name.

The code begins by importing the SQLDatabase class from the llama_index.core module.

Then, an instance of the SQLDatabase class is created, named sql_database.

The SQLDatabase instance is initialized with two arguments: engine and include_tables.

engine is a previously defined variable that represents the database engine to be used.

include_tables is a list of tables to be included in the database, in this case, only the "bank" table.

# Building the SQL query engine

In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(sql_database)

Create the SQL query engine using the NLSQLTableQueryEngine function by providing it with the LlamaIndex SQL database object. 

The code begins by importing the NLSQLTableQueryEngine class from the llama_index.core.query_engine module.

Then, an instance of the NLSQLTableQueryEngine class is created, named query_engine.

This instance is initialized with the sql_database parameter, which presumably represents a SQL database.

In [ ]:
response = query_engine.query("Which is the longest running campaign?")

print(response.response)

Ask the question from the query engine about the “bank” table in the natural language. 

The first line calls the query method of the query_engine object with the question "Which is the longest running campaign?".

The query method presumably sends this question to a database or search engine and returns the result.

This result is stored in the variable response.

The second line prints the response attribute of the response object, which likely contains the answer to the query.

In response, we will get the answer to your query in natural languages as "The longest running campaign in the database has a duration of 4918 days."

In [ ]:
response = query_engine.query("Which type of job has the most housing loan?")

print(response.response)

Let's ask a complex question.

"The job type with the most housing loans is 'admin.' with 5559 housing loans. This is followed by 'blue-collar' with 4710 housing loans and 'technician' with 3616 housing loans. Other job types with significant housing loans include 'services', 'management', 'retired', 'entrepreneur', and 'self-employed'." 

The above answer is precise, with additional information. 

In [ ]:
print(response.metadata)

To check what is going on on the back end, we will print the metadata. 

The code is written in Python, a high-level, interpreted programming language.

The print() function is a built-in Python function used to output data to the console.

response is an object that presumably holds some data, including a metadata attribute.

response.metadata is accessing the metadata attribute of the response object.

The code will print the value of response.metadata to the console.

In [ ]:
{'d4ddf03c-337e-4ee6-957a-5fd2cfaa4b1c': {}, 'sql_query': "SELECT job, COUNT(housing) AS housing_loan_count\nFROM bank\nWHERE housing = 'yes'\nGROUP BY job\nORDER BY housing_loan_count DESC;", 'result': [('admin.', 5559), ('blue-collar', 4710), ('technician', 3616), ('services', 2050), ('management', 1490), ('retired', 892), ('entrepreneur', 779), ('self-employed', 740), ('unemployed', 557), ('housemaid', 540), ('student', 471), ('unknown', 172)], 'col_keys': ['job', 'housing_loan_count']}

The code is a SQL query embedded in a Python dictionary.

The key 'sql_query' contains the actual SQL command.

"SELECT job, COUNT(housing) AS housing_loan_count" selects the 'job' column and counts the 'housing' column.

"FROM bank" specifies the table 'bank' from which the data is selected.

"WHERE housing = 'yes'" filters the data to only include rows where 'housing' is 'yes'.

"GROUP BY job" groups the data by the 'job' column.

"ORDER BY housing_loan_count DESC" sorts the data in descending order by the count of 'housing'.

The 'result' key in the dictionary stores the result of the SQL query.

The 'col_keys' key in the dictionary stores the column names of the result.

As we can see, GPT-4o first generates the SQL query, runs the query to get the result, and uses the result to generate the response. This multi-step process is achieved through two lines of code.

In [ ]:
engine.close()

Close the engine when you are done with the project. 

This is a single line of Python code that calls the close() method on an object named engine.

The close() method is commonly used to close a connection or a file.

In this context, it's likely that engine is a database engine or a connection to a database.

By calling engine.close(), the code is closing the connection to the database to free up resources.